In [2]:
import pandas as pd
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [1]:
!pip3 install mlxtend

     |████████████████████████████████| 1.3 MB 1.8 MB/s eta 0:00:01


In [3]:
down = pd.read_csv('./Data_OTT/movie_down.csv', engine='python', encoding='cp949')
down.head()

,customer_id,item_id,down_date,down_year,down_month,down_weekday
0,C5001,I-1038,2015-01-08,2015,2015M01,Thu
1,C5001,I-1003,2015-01-17,2015,2015M01,Sat
2,C5001,I-1017,2015-01-17,2015,2015M01,Sat
3,C5001,I-1028,2015-01-19,2015,2015M01,Mon
4,C5001,I-1036,2015-01-20,2015,2015M01,Tue


In [9]:
down_test = down[down['customer_id'].isin(['C5001', 'C5002', 'C5003', 'C5004',
                                          'C5005', 'C5006', 'C5007', 'C5008',
                                          'C5009', 'C5010'])]
down_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492 entries, 0 to 78883
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   492 non-null    object
 1   item_id       492 non-null    object
 2   down_date     492 non-null    object
 3   down_year     492 non-null    int64 
 4   down_month    492 non-null    object
 5   down_weekday  492 non-null    object
dtypes: int64(1), object(5)
memory usage: 26.9+ KB


In [26]:
all_list = []
id_list = []

customer_id = list(pd.unique(down['customer_id']))

for i in customer_id:
    all_list.append(list(pd.unique(down[down['customer_id'] == i]['item_id'])))

len(all_list)

3277

In [27]:
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = all_list)

df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)
df_asso

,I-1001,I-1002,I-1003,I-1004,I-1005,I-1006,I-1007,I-1008,I-1009,I-1010,I-1011,I-1012,I-1013,I-1014,I-1015,I-1016,I-1017,I-1018,I-1019,I-1020,I-1021,I-1022,I-1023,I-1024,I-1025,I-1026,I-1027,I-1028,I-1029,I-1030,I-1031,I-1032,I-1033,I-1034,I-1035,I-1036,I-1037,I-1038,I-1039,I-1040,...,I-1055,I-1056,I-1057,I-1058,I-1059,I-1060,I-1061,I-1062,I-1063,I-1064,I-1065,I-1066,I-1067,I-1068,I-1069,I-1070,I-1071,I-1072,I-1073,I-1074,I-1075,I-1076,I-1077,I-1078,I-1079,I-1080,I-1081,I-1082,I-1083,I-1084,I-1085,I-1086,I-1087,I-1088,I-1089,I-1090,I-1091,I-1092,I-1093,I-1094
0,False,True,True,False,False,True,False,False,False,True,False,False,False,True,True,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,True,True,True,True,False,...,True,True,False,False,False,False,True,False,False,False,True,False,True,False,False,False,True,False,False,False,False,True,True,False,True,True,True,False,False,False,True,True,False,True,False,False,False,True,True,False
1,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,True
2,False,False,False,True,True,False,True,False,True,False,True,False,True,False,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,False,True,True,True,...,True,True,True,False,True,True,True,False,True,False,False,True,False,False,False,False,False,True,True,False,False,False,False,True,True,True,False,False,True,False,False,True,False,False,False,False,True,False,True,False
3,False,False,True,True,True,True,False,True,True,False,True,False,True,True,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,True,True,True,False,False,True,False,...,True,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,False,True,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True
4,True,True,True,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,True,True,False,False,True,...,True,True,True,True,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,True,False,False,True,True,False,False,True,False,False,True,False,False,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,True,False,False,True,False,True,False,False,True,False,False,False
3273,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [53]:
v_min_sup = 0.2

df_freq = apriori(df_asso, min_support=v_min_sup, use_colnames=True)
df_freq.sort_values(by = 'support', ascending = False)

,support,itemsets
23,0.425999,(I-1048)
27,0.415929,(I-1052)
32,0.406469,(I-1057)
63,0.404944,(I-1094)
62,0.396704,(I-1093)
...,...,...
0,0.208727,(I-1002)
9,0.208422,(I-1015)
64,0.204150,"(I-1044, I-1048)"
65,0.203540,"(I-1044, I-1052)"


In [67]:
df_asso_rule = association_rules(df_freq, metric = 'confidence', min_threshold=0.2)
df_asso_rule.sort_values(by = 'lift', ascending = False).head(50)

df_asso_rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(I-1044),(I-1048),0.361001,0.425999,0.204150,0.565511,1.327493,0.050364,1.321095
1,(I-1048),(I-1044),0.425999,0.361001,0.204150,0.479226,1.327493,0.050364,1.227019
2,(I-1044),(I-1052),0.361001,0.415929,0.203540,0.563821,1.355569,0.053389,1.339062
3,(I-1052),(I-1044),0.415929,0.361001,0.203540,0.489362,1.355569,0.053389,1.251373
4,(I-1048),(I-1052),0.425999,0.415929,0.219408,0.515043,1.238295,0.042222,1.204376
...,...,...,...,...,...,...,...,...,...
605,(I-1060),"(I-1063, I-1059, I-1062, I-1064, I-1061)",0.382057,0.216356,0.210558,0.551118,2.547270,0.127898,1.745768
606,(I-1059),"(I-1063, I-1060, I-1062, I-1064, I-1061)",0.375954,0.222154,0.210558,0.560065,2.521062,0.127039,1.768092
607,(I-1062),"(I-1063, I-1060, I-1059, I-1064, I-1061)",0.378700,0.221544,0.210558,0.556003,2.509673,0.126660,1.753292
608,(I-1064),"(I-1063, I-1060, I-1059, I-1062, I-1061)",0.371376,0.219103,0.210558,0.566968,2.587680,0.129189,1.803324
